In [127]:
import math
import numpy as np
import random

In [128]:
DEPOT = (4.4184, 114.0932)

CUSTOMERS = [
    (4.3555, 113.9777, 5),
    (4.3976, 114.0049, 8),
    (4.3163, 114.0764, 3),
    (4.3184, 113.9932, 6),
    (4.4024, 113.9896, 5),
    (4.4142, 114.0127, 8),
    (4.4804, 114.0734, 3),
    (4.3818, 114.2034, 6),
    (4.4935, 114.1828, 5),
    (4.4932, 114.1322, 8)
]

VEHICLES = [
    {"type": "A", "capacity": 25, "cost_per_km": 1.2},
    {"type": "B", "capacity": 30, "cost_per_km": 1.5}
]

In [129]:

def euclidean_distance(p1, p2):
    return 100 * math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)


# Compute the distance in matrix form

def calculate_distances():
    
    '''
    row 1 -->  customer 1
    row 2 -->  customer 2
    row 3 -->  customer 3
    ...
    row 10 -->  customer 10
    row 11 -->  Depot
    '''

    distances = np.zeros((len(CUSTOMERS) + 1, len(CUSTOMERS) + 1))
    
    # Compute the distances between customers with index [0 to 9]
    for i in range(len(CUSTOMERS)):
        for j in range(i+1, len(CUSTOMERS)):
            
            dist = euclidean_distance(CUSTOMERS[i][:2], CUSTOMERS[j][:2])  
            distances[i][j] = distances[j][i] = dist #invert insertion
    
    
    # The distance between depot and customers are computed in the last row
    for i in range(distances.shape[1]- 1):
        distances[-1][i] = distances[i][-1] = euclidean_distance(DEPOT, CUSTOMERS[i][:2])
        
    return distances
    

In [130]:
calculate_distances().shape

(11, 11)

In [131]:
VEHICLES = [
    {"type": "A", "capacity": 25, "cost_per_km": 1.2},
    {"type": "B", "capacity": 30, "cost_per_km": 1.5}
]


def pick_vehicle(remaining_demand, vehicles):
    
    max_capacity_vehicle = max(vehicles, key=lambda x: x["capacity"])
    
    # If the remaining demand is more than the vehicle with the largest capacity, deploy the vehicle with largest capacity for the next route
    if remaining_demand > max_capacity_vehicle["capacity"]:
        return max_capacity_vehicle
    
    else:
        suitable_vehicles = [vehicle for vehicle in vehicles if vehicle["capacity"] >= remaining_demand]
        
        if suitable_vehicles:
    
    #If the remaining demand is less than the capacity of the largest vehicle, 
    #deploy the vehicle with the smallest capacity that can meet the remaining demand to avoid underutilization.
            
            return min(suitable_vehicles, key=lambda x: x["capacity"])
        else:
            return None

## ACO

In [132]:

# ACO parameters
NUM_ANTS = 50

MAX_ITERATIONS = 1000 
ALPHA = 1  # Pheromone importance
BETA = 1   # Heuristic information importance
RHO = 0.2  # Evaporation rate
Q = 5    # Pheromone deposit factor
INITIAL_PHEROMONE = 1.0


In [133]:

class ACO:
    
    def __init__(self):
        self.num_nodes = len(CUSTOMERS) + 1
        
        self.distances = calculate_distances() # Matrix representation of the distance
        
        self.pheromones = np.full((self.num_nodes, self.num_nodes), INITIAL_PHEROMONE) # Matrix representation of pheromone
        
        self.best_solution = None
        self.best_cost = float('inf')
        self.total_demand = sum(customer[2] for customer in CUSTOMERS)
    


    def construct_ant_solution(self):
        
        unvisited = set(range(0, self.num_nodes-1)) # Represent customer nodes
        solution = []
        current_route = []
        
        remaining_demand = self.total_demand
        
        current_vehicle = pick_vehicle(remaining_demand, VEHICLES)
        remaining_capacity = current_vehicle["capacity"]
        current_node = len(CUSTOMERS)  # Start at the depot
        

        while unvisited:
            
            next_node = self.select_next_node(current_node, unvisited, remaining_capacity)
            
            if next_node is None:
                if current_route:
                    
                    # The route of the current vehicle: depot + current_route + depot
                    solution.append((current_vehicle, current_route))
                
                # Return to depot and start a new route    
                current_route = []
                current_vehicle = pick_vehicle(remaining_demand, VEHICLES)
              
               
                remaining_capacity = current_vehicle["capacity"]
                current_node = len(CUSTOMERS)  # Restart at the depot   
                
            else:
               
                current_route.append(next_node)
                remaining_capacity -= CUSTOMERS[next_node][2]
                remaining_demand -= CUSTOMERS[next_node][2]
                unvisited.remove(next_node)  ## REMOVE the visited node
                current_node = next_node

        if current_route:
            solution.append((current_vehicle, current_route))
           
        return solution
    
    
    

    def select_next_node(self, current_node, unvisited, remaining_capacity):
        
        probabilities = []
        
        for node in unvisited:
           
            if CUSTOMERS[node][2] <= remaining_capacity:  
                
                pheromone = self.pheromones[current_node][node]
                distance = self.distances[current_node][node]  
                probability = (pheromone ** ALPHA) * ((1 / distance) ** BETA)
                probabilities.append((node, probability))  # Customer num and prob
                

        # if the list is empty, the neighbouring nodes are all visited OR no capacity left
        if not probabilities:     
            return None

        total = sum(prob for node, prob in probabilities)
        
        # Include denominator
        normalized_probabilities = [(node, prob / total) for node, prob in probabilities]
        
        
        # Pick the path randomly by using the computed probability as weight (higher probablity has a higher chance to get picked)
        return random.choices(normalized_probabilities, weights=[prob for node, prob in normalized_probabilities])[0][0]
    

                    

    def update_pheromones(self, solutions):
        # Evaporation
        self.pheromones *= (1 - RHO)
        
        # Deposit
        for solution in solutions:
            cost = self.calculate_total_cost(solution)
        
            for vehicle, route in solution:
                for i in range(len(route) - 1):
                    self.pheromones[route[i]][route[i+1]] += Q / cost
                    
                # Add pheromone for the edges connecting to depot
                self.pheromones[-1][route[0]] += Q / cost
                self.pheromones[route[-1]][-1] += Q / cost
                
                
                

    def calculate_total_cost(self, solution):
        
        total_cost = 0
        
        for vehicle, route in solution:
            
            route_distance = sum(self.distances[route[i]][route[i+1]] for i in range(len(route)-1))
            
            # Add depot to the first customer and last customer to depot
            route_distance += self.distances[-1][route[0]] + self.distances[route[-1]][-1]
            
            total_cost += route_distance * vehicle["cost_per_km"]
            
        return total_cost
    
    
   
    
    # Run iteration with early stopping
    def run(self):
        best_costs = []  
        no_improvement_limit = 50  # The limit for the number of iterations without improvement
        
        for iteration in range(MAX_ITERATIONS):
            ant_solutions = [self.construct_ant_solution() for _ in range(NUM_ANTS)]
            self.update_pheromones(ant_solutions)
            
            #print(len(ant_solutions))  ## 50
            
            iteration_best = min(ant_solutions, key=lambda x: self.calculate_total_cost(x))
            iteration_best_cost = self.calculate_total_cost(iteration_best)
            
            if iteration_best_cost < self.best_cost:
                self.best_solution = iteration_best
                self.best_cost = iteration_best_cost

            print(f"Iteration {iteration + 1}: Best cost = {self.best_cost:.2f}")

            # Append the best cost of the current iteration to the list
            best_costs.append(self.best_cost)
            
            # Maintain the list to keep only the last `no_improvement_limit` costs
            if len(best_costs) > no_improvement_limit:
                best_costs.pop(0)
            
            # Check if all elements in `best_costs` are the same
            if len(best_costs) == no_improvement_limit and all(cost == best_costs[0] for cost in best_costs):
                print(f"Early stopping at iteration {iteration + 1}: No improvement in the last {no_improvement_limit} iterations.")
                break

        return self.best_solution, self.best_cost

In [134]:
aco = ACO()
best_solution, best_cost = aco.run()

Iteration 1: Best cost = 158.99
Iteration 2: Best cost = 147.97
Iteration 3: Best cost = 147.97
Iteration 4: Best cost = 145.73
Iteration 5: Best cost = 145.73
Iteration 6: Best cost = 145.73
Iteration 7: Best cost = 145.73
Iteration 8: Best cost = 145.73
Iteration 9: Best cost = 145.73
Iteration 10: Best cost = 145.73
Iteration 11: Best cost = 145.73
Iteration 12: Best cost = 145.73
Iteration 13: Best cost = 145.73
Iteration 14: Best cost = 140.37
Iteration 15: Best cost = 140.37
Iteration 16: Best cost = 140.37
Iteration 17: Best cost = 136.32
Iteration 18: Best cost = 136.32
Iteration 19: Best cost = 136.32
Iteration 20: Best cost = 136.32
Iteration 21: Best cost = 136.32
Iteration 22: Best cost = 136.32
Iteration 23: Best cost = 136.32
Iteration 24: Best cost = 136.32
Iteration 25: Best cost = 136.32
Iteration 26: Best cost = 136.32
Iteration 27: Best cost = 136.32
Iteration 28: Best cost = 133.94
Iteration 29: Best cost = 133.94
Iteration 30: Best cost = 133.94
Iteration 31: Best 

In [135]:
def format_solution(solution, cost, matrix_distance):
    output = []
    total_distance = 0
    
    for i, (vehicle, route) in enumerate(solution, 1):
        
        
        # Compute the route distance between customer
        distance = sum(matrix_distance[route[j]][route[j+1]] for j in range(len(route) - 1))
        
        # Add the distance of depot to first customer and last customer to depot
        distance += matrix_distance[-1][route[0]] + matrix_distance[route[-1]][-1]
        
        # Sum up the demand delivered for each vehicle
        demand = sum(CUSTOMERS[node][2] for node in route)
        
        
        route_cost = distance * vehicle["cost_per_km"]
        total_distance += distance
        
        route_str = f"Depot -> {' -> '.join(map(str, route))} -> Depot"
        output.append(f"Vehicle {i} (Type {vehicle['type']}):")
        output.append(f"Round Trip Distance: {distance:.2f} km, Cost: RM {route_cost:.2f}, Demand: {demand}")
        output.append(route_str)
        output.append("")
    
    output.append(f"Total Distance = {total_distance:.2f} km")
    output.append(f"Total Cost = RM {cost:.2f}")
    
    
    return "\n".join(output)


In [136]:
matrix_distance = calculate_distances()
print(format_solution(best_solution, best_cost, matrix_distance))

Vehicle 1 (Type B):
Round Trip Distance: 51.09 km, Cost: RM 76.64, Demand: 30
Depot -> 5 -> 6 -> 9 -> 8 -> 7 -> Depot

Vehicle 2 (Type B):
Round Trip Distance: 38.20 km, Cost: RM 57.31, Demand: 27
Depot -> 1 -> 4 -> 0 -> 3 -> 2 -> Depot

Total Distance = 89.30 km
Total Cost = RM 133.94
